In [1]:
!pip install transformers

In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.9 MB/s eta 0:00:00


In [3]:
from transformers import XLNetTokenizer, XLNetModel

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state


spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

In [ ]:
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Example sentences
sentence1 = "a cruise autonomous vehicle cruise av operating in manual mode was traveling northbound on th street in the rightmost lane between folsom street and south van ness avenue at the same time a model tesla traveling in the adjacent center lane alongside the cruise av attempted to change lanes in front of the cruise av over a solid white line into the rightmost lane in which the cruise av was traveling during this maneuver the model tesla made contact with the cruise av s driver side rear wheel and wheel well and driver side rear passenger door damaging the tesla s front passenger side wheel well the parties exchanged information and there were no reported injuries"
sentence2 = "a tesla vehicle in autonomous mode was driving down a road when it encountered a car in the opposite lane the tesla vehicle attempted to avoid a collision by swerving to the right but the car in the opposite lane did not stop and continued to move forward the tesla vehicle was able to avoid a direct collision but the car in the opposite lane made contact with the tesla vehicle s front sensors the car in the opposite lane did not stop and continued to travel down the road no injuries were reported and the police were not called to the scene"

# Tokenize and encode sentences
inputs1 = tokenizer(sentence1, return_tensors="pt", padding=True, truncation=True)
inputs2 = tokenizer(sentence2, return_tensors="pt", padding=True, truncation=True)

# Obtain model embeddings
with torch.no_grad():
    embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity_matrix = cosine_similarity(embeddings1, embeddings2)
similarity_score = similarity_matrix[0][0]

print(f"Similarity Score: {similarity_score}")

Similarity Score: 0.9930583834648132


In [4]:
import pandas as pd
import re
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
def DescriptionCleaning(df, columns, name):
  columns_list = []
  columns_list.append(columns)
  columns_list.append(name)
  print(columns_list)
  df = df[columns_list]
  df=df.dropna()

  labelRaw = list(df[columns])
  label = []
  for text in labelRaw:
    text = str(text)
    temp = re.sub('[^a-zA-Z]+', ' ', text)
    temp = temp.lower().strip()
    label.append(temp)

  df[columns]=label

  return df

In [6]:
file_path = "/content/Accidents_data_Processed.csv"
df = pd.read_csv(file_path)
df = DescriptionCleaning(df, "Description", "BUSINESS NAME")

['Description', 'BUSINESS NAME']


In [7]:
file_path = "/content/Image to text output.xlsx"
manual_driving_df = pd.read_excel(file_path)
df1 = DescriptionCleaning(manual_driving_df, "Llava output", "Video name")

['Llava output', 'Video name']


In [ ]:
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['Llava output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.99:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

In [10]:
outputDF = pd.DataFrame(data)
outputDF.to_excel('XLNET_similarity_score.xlsx', sheet_name = 'LLava_comparision')

In [ ]:
df1 = DescriptionCleaning(manual_driving_df, "VQA and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['VQA and LLaMA Output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.95:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

In [ ]:
outputDF = pd.DataFrame(data)

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/XLNET_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'VQA_and_LLama')
writer.close()

In [ ]:
df1 = DescriptionCleaning(manual_driving_df, "BLIPVQA 13B and LLaMA Output", "Video name")
count = 0
data = []
obj = {}
# Creating Empty DataFrame and Storing it in variable df
outputDF = pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df1)):
        obj = {"business_name": "", "prompt1": "", "video_name": "", "prompt2": "", "score": 0}
        row1 = df.iloc[i]
        row2 = df1.iloc[j]
        if i != j:
            prompt1 = row1['Description']
            prompt2 = row2['BLIPVQA 13B and LLaMA Output']

            # Tokenize and encode sentences
            inputs1 = tokenizer(prompt1, return_tensors="pt", padding=True, truncation=True)
            inputs2 = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True)
            # Obtain model embeddings
            with torch.no_grad():
                embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
                embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)
            # Calculate cosine similarity
            similarity_matrix = cosine_similarity(embeddings1, embeddings2)
            similarity_score = similarity_matrix[0][0]
            obj["business_name"] = row1["BUSINESS NAME"]
            obj["prompt1"] = prompt1
            obj["video_name"] = row2["Video name"]
            obj["prompt2"] = prompt2
            obj["score"] = similarity_score
            data.append(obj)
            if similarity_score > 0.95:
              count = count + 1
              #print(f"Similarity between Prompt {i+1} and Prompt {j+1}: {similarity_score}")
              #print(prompt1)
              #print(prompt2)
              print(i+1)

print(count)

In [ ]:
outputDF = pd.DataFrame(data)

In [ ]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

path = r"/content/XLNET_similarity_score.xlsx"

book = load_workbook(path)
writer = pd.ExcelWriter(path, engine = 'openpyxl')
writer.book = book

outputDF.to_excel(writer, sheet_name = 'BLIPVQA_13B_and_LLaMA')
writer.close()